In [ ]:
import requests
import bs4
from toolz.itertoolz import mapcat, unique
from toolz.dicttoolz import merge
import logging
from dask.distributed import Client
import pandas as pd
import sqlite3

In [ ]:
logging.basicConfig()
logger = logging.getLogger('get_bas_videos')
logger.setLevel(logging.DEBUG) # Do not chain this to previous statement

In [ ]:
conn = sqlite3.connect("<RSS Feeds DB>")

In [ ]:
df = pd.read_sql_query("select * from news where feedid = 30;", conn)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.to_clipboard(sep='\t', index=False)

In [ ]:
def extract_links(html):
    logger.debug(len(html))
    soup = bs4.BeautifulSoup(html)
    anchors = soup.find_all("a")
    anchors_list = [ a.get('href', '') for a in anchors]
    return anchors_list

In [ ]:
links_list = [extract_links(html) for html in list(df['description'])]

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

### Which is equivalent to
```
flat_list = []
for sublist in l:
    for item in sublist:
        flat_list.append(item)
```

In [ ]:
flat_list = flatten(links_list)

In [ ]:
flat_list_undups = [x for x in unique(flat_list)]

In [ ]:
flat_list_undups

In [ ]:
def unshorten_url(s):
    logger.debug(s['links'])
    
    result = ''
    try:
        r = requests.head(s['links'], allow_redirects=True)
        result = r.url
    except:
        pass
#    r = requests.head(s['links'], allow_redirects=True)
    return result

In [ ]:
links_df = pd.DataFrame({ 'links': flat_list_undups})

In [ ]:
links_df['unshortened_url'] = links_df.apply( lambda x: unshorten_url(x), axis=1 )

In [ ]:
links_df.info()

In [ ]:
links_df.head()

In [ ]:
links_df.drop_duplicates(['unshortened_url'], inplace =True)

In [ ]:
links_df.to_clipboard(sep='\t', index=False)

In [ ]:
links_df.to_csv('C://MMM/twitter_links.csv')